# Demonstrate Using MLflow and TensorBoard with CIFAR-10 dataset

## Preparing Required Libraries

In [1]:
!pip install -q -U keras-tuner

In [2]:
import datetime
import IPython
import kerastuner as kt
import mlflow
import numpy as np
import os
import pickle
import random
import tarfile
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.keras import datasets, layers, models
from tensorboard.plugins.hparams import api as hp

In [3]:
# set seed for reproducibility
SEED = 1234
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

/home/ai/.pyenv/versions/3.7.9/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Preparing CIFAR-10 Dataset

In [4]:
print(f'Setting up MLflow experiment...')
experiment_name = 'cifar10-train'
mlflow_tracking_uri = os.getenv('MLFLOW_TRACKING_URI')
print(f'MLflow tracking uri: {mlflow_tracking_uri}')
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment(experiment_name)

print(f'Extracting Cifar10 dataset...')
tar = tarfile.open('../data/cifar-10-python.tar.gz')
tar.extractall(path='../data')
tar.close()

Setting up MLflow experiment...
MLflow tracking uri: http://mlflow:5000
Extracting Cifar10 dataset...


## Loading Training/Testing Images

In [5]:
# reference: https://github.com/tensorflow/tensorflow/blob/9011878d87bdeff932e10e2b2d35570be5ef739e/tensorflow/python/keras/datasets/cifar.py#L26
def load_batch(fpath, label_key='labels'):
    """Internal utility for parsing CIFAR data.
    Arguments:
      fpath: path the file to parse.
      label_key: key for label data in the retrieve
          dictionary.
    Returns:
      A tuple `(data, labels)`.
    """
    with open(fpath, 'rb') as f:
        d = pickle.load(f, encoding='bytes')
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode('utf8')] = v
        d = d_decoded
    data = d['data']
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels


# reference: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/keras/datasets/cifar10.py#L32
def load_data():
    dir_name = '../data/cifar-10-batches-py'

    # load train data
    num_train_samples = 50000

    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = f'{dir_name}/data_batch_{i}'
        (x_train[(i - 1) * 10000:i * 10000, :, :, :],
         y_train[(i - 1) * 10000:i * 10000]) = load_batch(fpath)

    # load test data
    fpath = f'{dir_name}/test_batch'
    x_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    if K.image_data_format() == 'channels_last':
        x_train = x_train.transpose(0, 2, 3, 1)
        x_test = x_test.transpose(0, 2, 3, 1)

    x_test = x_test.astype(x_train.dtype)
    y_test = y_test.astype(y_train.dtype)

    return (x_train, y_train), (x_test, y_test)

In [6]:
print(f'Loading train/test images...')
(train_images, train_labels), (test_images, test_labels) = load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# split validation data from testing data
TESTING_DATA_SIZE = 100
test_images, valid_images  = test_images[:TESTING_DATA_SIZE], test_images[TESTING_DATA_SIZE:]
test_labels, valid_labels  = test_labels[:TESTING_DATA_SIZE], test_labels[TESTING_DATA_SIZE:]
print(f'Training data size: {len(train_images)}')
print(f'Validation data size: {len(valid_images)}')
print(f'Testing data size: {len(test_images)}')

Loading train/test images...
Training data size: 50000
Validation data size: 9900
Testing data size: 100


## Hyperparameter Tuning, Model Training and Evaluation

This notebook would demonstrate using hyperparameter tuning for a simple CNN network model, and you can later compare different experiment results using MLflow and TensorBoard.

### Define the Model

reference: https://www.tensorflow.org/tutorials/keras/keras_tuner

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

- By using a model builder function
- By subclassing the HyperModel class of the Keras Tuner API

You can also use two pre-defined HyperModel classes - HyperXception and HyperResNet for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [7]:
def model_builder(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-128
    hp_units = hp.Int('units', min_value=32, max_value=128, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    model.add(layers.Dense(10))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

### Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - 
`RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`.

In this tutorial, you use the `Hyperband` tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing $1 + log_{factor}($max_epochs$)$ and rounding it up to the nearest integer.

```
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='../logs/',
                     project_name='keras_tuner')
```

Before running the hyperparameter search, define a callback to clear the training outputs at the end of every training step.

In [8]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='../logs/',
                     project_name='keras_tuner')

log_dir = '../logs/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
tuner.search(train_images,
             train_labels,
             epochs=10,
             validation_data=(valid_images, valid_labels),
             callbacks=[ClearTrainingOutput(), tensorboard_callback])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 18 Complete [00h 03m 12s]
val_accuracy: 0.7083838582038879

Best val_accuracy So Far: 0.7136363387107849
Total elapsed time: 00h 27m 26s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 128 and the optimal learning rate for the optimizer
is 0.001.



### Model Training and Evaluation

In [10]:
mlflow.tensorflow.autolog() # enable autologging for MLflow

with mlflow.start_run():
    # Build the model with the optimal hyperparameters and train it on the data
    model = tuner.hypermodel.build(best_hps)
    
    log_dir = '../logs/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    model.fit(train_images,
              train_labels,
              epochs=10,
              validation_data=(test_images, test_labels),
              callbacks=[tensorboard_callback])
    
    loss, accuracy = model.evaluate(test_images, test_labels)
    print(f'testing loss: {loss}')
    print(f'testing accuracy: {accuracy}')

Epoch 1/10
1563/1563 [==============================] - 31s 20ms/step - loss: 1.4634 - accuracy: 0.4714 - val_loss: 1.1555 - val_accuracy: 0.6100
Epoch 2/10
1563/1563 [==============================] - 33s 21ms/step - loss: 1.0959 - accuracy: 0.6150 - val_loss: 0.9078 - val_accuracy: 0.6600
Epoch 3/10
1563/1563 [==============================] - 40s 25ms/step - loss: 0.9362 - accuracy: 0.6707 - val_loss: 0.8019 - val_accuracy: 0.7200
Epoch 4/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.8339 - accuracy: 0.7075 - val_loss: 0.7586 - val_accuracy: 0.7400
Epoch 5/10
1563/1563 [==============================] - 36s 23ms/step - loss: 0.7598 - accuracy: 0.7326 - val_loss: 0.7546 - val_accuracy: 0.7200
Epoch 6/10
1563/1563 [==============================] - 40s 26ms/step - loss: 0.6901 - accuracy: 0.7555 - val_loss: 0.7672 - val_accuracy: 0.7100
Epoch 7/10
1563/1563 [==============================] - 37s 24ms/step - loss: 0.6348 - accuracy: 0.7767 - val_loss: 0.8199 -